# Task 1 Parsing Raw Text Files

#### Name: Bolin Yang


Date: 20190412

Environment: Python 3.7 and Jupyter notebook
Libraries used: 
* json
* re (for regular expression, included in Anaconda Python 3.7) 


## 1. Introduction
This assessment touches the very first step of analyzing textual data, i.e., extracting data from unstructured text files. ​Each student is provided with a data-set that contains several job postings (please find your own file from ​task1.rar​, i.e., ​<your_student_number>.dat​). Each data-set contains information about the units, e.g., unit code, title, synopsis, pre_requisites, prohibitions,etc . Your task is to extract the data and transform the data to the ​XML ​and ​JSON ​format. ​Please note that the ​re ​and ​json packages ​in​ Python​ are the only packages that you are allowed to use in this task and the following must be performed to complete the assessment.

## 2. Import libraries

In [1]:
import re
import json

## 3.Read file
The steps are follows:
* Read file 
* split the contain of each unit

In [2]:
'''
Open and read the text and split them by "<!-- /.content-inner__main --></div>" for each unit, so it is easily to search
and extract data in each unit ,then store the extracted data into each unit section
'''

f = open ("Data_file/units_data.txt")
line = f.read()

#split units and count the number of units
units=line.split("<!-- /.content-inner__main --></div>")
unit_block=units[:-1]
unit_count=len(unit_block)
print(unit_count)


400


## 3. Extract data 
The data is extracted as follow:
* unit code
* title
* synopisi
* pre-requisites
* prohibitions
* requirements
* Outputs
* Chief-examiners

In [3]:
#find and extract unitcode
'''
each unit code is followed by the setence "<span class=\"unitcode\">", so extract unit code and store it as 
the key of dict as the same time
'''
unitcode=[]
for each in unit_block:
    if re.search("<span class=\"unitcode\">",each):
        unit=re.search("(?P<unitcode>[A-Z]{3}[0-9]{4})",each).groupdict()
        unitcode.append(unit)
print(unitcode)

In [4]:
#find and extract title
'''
the method of extracting title of each unit is same as the unitcode 
'''
title=[]
for each in unit_block:
    if re.search("<span class=\"unitcode\">",each):
        unitle=re.search("(?P<title></span> -.+<span class=\"hbk-archive-only\">)",each).groupdict()
        #print(unitle)
        temtit = unitle["title"].replace('<span class=\"hbk-archive-only\">', '.').replace('</span> - ', '')
        unitle["title"]=temtit
        title.append(unitle)
#print(title)

In [5]:
#find and extract Synopsis
'''
Since some synopsis of unit is na , so add if condiction to judge it firstly, then extract synopsis according the
sentence before and after synopsis, then delete the symbol we don't need
'''
synop=[]
item={}
for each in unit_block:
    if re.search("Synopsis</h2>", each):
        if re.search("Synopsis</h2>\s<div>\s</div>",each):
            item["synopsis"]="NA"
            synop.append(item)
        else:
            parts_syno=re.search('(?P<synopsis>Synopsis</h2>\s<div>\s<p>.+</p>)', each).groupdict()
            synop.append(parts_syno)
    for each in synop:
        part_syno = each["synopsis"].replace('Synopsis</h2>\n<div>\n<p>', '').replace('</p>','')
        each["synopsis"]=part_syno
#print(synop)

In [6]:
## find and extract Pre-requisites
''' define a function for judge if the number of value is not only one 
 if  number of unitcodes in each prerequisites >1, then store them into a set
 to make sure that each unitcode in value is unique'''

def deal_codes(name,names,block):          #block is a list
    item={}
    items={}
    result=[]
    blocks=set(block)
    for each in blocks:
        if len(blocks) > 1:
            result.append(each)
            item[name]=result
            items[names]=item
        else:
            item[name]=each
            items[names]=item
    return items                 #item is dict

In [7]:
#find and extract Pre-requisites, 
'''there are three situations
 1,only Pre-requisites; 2,Pre-requisites and Co-requisites; 3,Only Co-requisites;4,non prerequisites
 the first and second situation can express in first "if"judgment statement'''

prerequisites=[]
for each in unit_block:
    item = {}
    items= {}
    if re.search('Prerequisites</p>\s<p>.+', each):     #find pre-
        #print("cf")
        if re.search("Prerequisites</p>\s<p>(.|\n)+</div>",each):         # only have Prerequisites and don't have co-
            #print("cc")
            parts=re.findall("Prerequisites</p>\s<p>(?:.|\n)+</div>\n</div>",each)
            #print(parts)
            if re.search("(?P<pre_requistic>[A-Z]{3}[0-9]{4})",parts[0]):
                info=re.findall("(?P<pre_requistic>[A-Z]{3}[0-9]{4})",parts[0])
                #print(info)

                if len(info) > 1:
                    results=deal_codes('pre_requistic','pre_requistics',info)
                    #print(results)
                    prerequisites.append(results)

                elif len(info) == 1:
                    item["pre_requistic"]=info[0]
                    items["pre_requistics"]=item
                    prerequisites.append(items)
            else: # find Prerequisites but don't have unitcodes
                item["pre_requistics"] = "NA"
                prerequisites.append(item)

    elif re.search("Co-requisites</p>\s<p>(.|\n)+</div>",each) :     #only after  Co-requisites sentence
        parts=re.findall("Co-requisites</p>\s<p>(?:.|\n)+</div>",each)
        infor = re.findall("(?P<pre_requistic>[A-Z]{3}[0-9]{4})", parts[0])

        if len(infor) > 1:
            results = deal_codes('pre_requistic','pre_requistics',infor)

            prerequisites.append(results)
        elif len(infor) == 1:
            item["pre_requistic"] = infor[0]
            items["pre_requistics"] = item
            prerequisites.append(items)
        else:                                               # find Prerequisites but don't have unitcodes
            item["pre_requistics"] = "NA"
            prerequisites.append(item)

    else:
        item["pre_requistics"] = "NA"
        prerequisites.append(item)


#print(prerequisites)
print(len(prerequisites))

400


In [8]:
#find and extract Prohibition
'''
The method of it is same as above, extract prohibition of each unit and store it as a dict as the same time
if the length of value is not only one ,the value should be a list
'''
prohibit = []
for each in unit_block:
    it = {}
    ite = {}
    if re.search('Prohibitions</p>\n<p>', each):
        sets = re.findall('Prohibitions</p>\n<p>(?:.|\n)+</p>\n</div>\n</div>', each)
        pro = re.findall('(?P<prohibisions>[A-Z]{3}[0-9]{4})', sets[0])
        #print(pro)
        if len(pro) > 1:
            proh = deal_codes('prohibision','prohibisions', pro)
            prohibit.append(proh)
        elif len(pro) == 1:
            it['prohibision'] = pro[0]
            ite['prohibisions'] = it
            prohibit.append(ite)
        else:
            it['prohibisions'] = "NA"
            prohibit.append(it)
    else:
        it['prohibisions'] = "NA"
        prohibit.append(it)

#print(prohibit)
print(len(prohibit))

400


In [9]:
#Requirements(assessment)
''' define a function to delete the symbol and store the value into a list if the value not only one in one unit
 split the content that should be in the next line by the symbol "</li>\n<li>"'''
def spl_inform(data):
    item={}
    items={}
    lit=[]
    if re.search('</li>\n<li>',data):       #data is str
        lists=data.split('</li>\n<li>')
        for each in lists:
            info = re.sub('</\w+>\n', ',', each)
            info = re.sub('<\w+>', '', info)
            info = re.sub(',+<h2 class="hbk-heading">Workload', '', info)
            info = re.sub(',+', ',', info)
            inform = info.replace(':,', '').replace('\n', '')
            lit.append(inform)
        item['requirement']=lit
        items['requirements']=item

    else:
        info = re.sub('</\w+>\n', ',', data)
        info = re.sub('<\w+>', '', info)
        info = re.sub(',+<h2 class="hbk-heading">Workload', '', info)
        info = re.sub(',+', ',', info)
        inform = info.replace(':,', '').replace('\n', '')
        item['requirement'] = inform
        items['requirements'] = item
    return items

In [10]:
Requirements=[]
for each in unit_block:
    item = {}
    if re.search('Assessment</h2>\n<div>(.|\n)+</div>\n<h2 class="hbk-heading">[A-Z]',each):            #find requirements
        #extract information of requirements
        if re.search('(?P<requirement>Assessment</h2>\n<div>(.|\n)+</div>\n<h2 class="hbk-heading">Workload)',each):
            find=re.search('(?P<requirement>Assessment</h2>\n<div>(.|\n)+</div>\n<h2 class="hbk-heading">Workload)',each).groupdict()
            #print(find)

        #split the content in assessmentand delete <>
        #if re.search('Assessment</h2>\n<div>\n<ul>\n<li>.+',find['requirements']):

        #There are two diff forms of requirements,the first one:
            if re.search('Assessment</h2>\n<div>\n<p>.+',find['requirement']):
                #print("a")
                inf=find['requirement'].replace('Assessment</h2>\n<div>\n<p>','')
                #print(inf)

            #split content in one requirements
                infs=spl_inform(inf)
                #print(infs)
                #item['requirements']=infs
                Requirements.append(infs)

            elif re.search('Assessment</h2>\n<div>\n<ul|ol>(.|\n)+', find['requirement']):
                mess = find['requirement'].replace('Assessment</h2>\n<div>\n<ul>', '')
                message = spl_inform(mess)
                Requirements.append(message)
        else:
            item["requirements"] = "NA"
            Requirements.append(item)
    else:
        item["requirements"] = "NA"
        Requirements.append(item)


#print(Requirements)
print(len(Requirements))


400


In [11]:
#find and extract output
'''
After finding the position and sentence before Outcome, ectract it and store it into adict ,but some sentences after 
the Outcomes we don't need ,so delete the symbols and sentences
'''
outcomes = []

for each in unit_block:
    listt = []
    item = {}
    items = {}
    if re.search('Outcomes</h2>\n<div>', each):
        # extract the information of outcomes
        if re.search('(?P<output>Outcomes</h2>\n<div>(.|\n)+</li>\n</ol>\n</div>)', each):
            finds = re.search('(?P<outcome>Outcomes</h2>\n<div>(.|\n)+</li>\n</ol>\n</div>)', each).groupdict()
            # print(finds)
            tex = re.sub('Outcomes(</?\w+>|\n)+(.+</p>\n)*', '', finds['outcome'])
            # print(tex)
            text = tex.replace('<ol princestart="0" start="1" type="1">\n<li>', '')
            texts = re.sub('<ol.+">\n', '',text)
            tt = re.sub('\n','',texts)
            tt = re.sub('</li>|</ol>','',tt)
            textt = tt.replace('</p></li>', ':')
            # print(textt)
            # print(text)
            if re.search(';</li>\n<li>', textt):  # there are more than one lines content
                lists = textt.split(';</li>\n<li>')
                # print(lists)
                for chunck in lists:
                    out = re.sub('</\w+>\n', ',', chunck)
                    out = re.sub('<\w+>', '', out)
                    outs = out.replace(',,</div>', '')
                    listt.append(outs)
                # print(listt)
                item['outcome'] = listt
                items['outcomes'] = item
                outcomes.append(items)
            else:
                texts = re.sub('</\w+>\n', '', textt)
                texts = re.sub('<\w+>', '', texts)
                textss = texts.replace('</div>', '')
                item['outcome'] = textss
                items['outcomes'] = item

                outcomes.append(items)
        else:
            items['outcomes'] = "NA"
            outcomes.append(items)
    else:
        items['outcomes'] = "NA"
        outcomes.append(items)

#print(outcomes)
print(len(outcomes))




400


In [12]:
#Chief-examiners
'''
There are two kinds of sentencs after examiners, coodinaters and unitguide,so there are two conditions to judge, and 
same way to extract data in each condition 
'''
examiners=[]
for each in unit_block:
    new=[]
    dit={}
    dits={}
    if re.search('Chief examiner\(s\)</p>(.|\n)+</a>\n<br/>\n</p>',each):
        # There are two sentences forms after examiners Coodinaters or Unitguides

        if re.search('Chief examiner\(s\)</p>(?:.|\n)+</a>\n(?:.|\n)*<br/>\n</p>\n<p class="hbk-highlight-heading">Coordinator\(s\)</p>',each):
            exam=re.findall('Chief examiner\(s\)</p>(?:.|\n)+</a>\n(?:.|\n)*<br/>\n</p>\n<p class="hbk-highlight-heading">Coordinator\(s\)</p>',each)
            exami = re.findall('(?P<chief_examiner>">.+</a>)', exam[0])
            if len(exami)>1:
                for each in exami:
                    ret=re.sub('">|</a>','',each)      # each is str
                    new.append(ret)
                dit['chief_examiner'] = new
                dits['chief_examiners'] = dit
                examiners.append(dits)
            else:
                examin=re.sub('">|</a>','',exami[0])
                dit['chief_examiner'] = examin
                dits['chief_examiners']= dit
                examiners.append(dits)
        elif re.search('Chief examiner\(s\)</p>(?:.|\n)+</a>\n<br/>\n</p>\n<p class="hbk-highlight-heading">Unit guides</p>',each):
            exams=re.findall('Chief examiner\(s\)</p>(?:.|\n)+</a>\n<br/>\n</p>\n<p class="hbk-highlight-heading">Unit guides</p>',each)
            if re.search('<p class="hbk-highlight-heading">Coordinator\(s\)</p>',exams[0]):
                pass
            else:
                exami = re.findall('(?P<chief_examiner>">.+</a>)', exams[0])
                if len(exami)>1:
                    for each in exami:
                        ret=re.sub('">|</a>','',each)      # each is str
                        new.append(ret)
                    dit['chief_examiner'] = new
                    dits['chief_examiners'] = dit
                    examiners.append(dits)
                else:
                    examin=re.sub('">|</a>','',exami[0])
                    dit['chief_examiner'] = examin
                    dits['chief_examiners']=dit
                    examiners.append(dits)
        else:
            dits['chief_examiners'] = "TBA"
            examiners.append(dits)
    else:
            dits['chief_examiners'] = "TBA"
            examiners.append(dits)

#print(examiners)
print(len(examiners))


400


## 4. Output for task1

* transmit the extract data into json
* tramsmit the extract data into xml

To json format

In [13]:
###change the name of unit from unitcode to @id
'''
Since the name of unitid we used is unitcode , we have to change it to @id and each unit in a dict , those unit in a list
'''
unitinform=[]
for each in unitcode:
    dictt={}
    dictt['@id']=each['unitcode']
    unitinform.append(dictt)
print(unitinform)
print(len(unitinform))


[{'@id': 'ATS1020'}, {'@id': 'MPH5218'}, {'@id': 'FIT2095'}, {'@id': 'LAW4108'}, {'@id': 'LAW4190'}, {'@id': 'APG5640'}, {'@id': 'MEC5889'}, {'@id': 'PGP5012'}, {'@id': 'ATS2837'}, {'@id': 'ASP2011'}, {'@id': 'MTE6885'}, {'@id': 'MED4200'}, {'@id': 'ARC3001'}, {'@id': 'BTS2301'}, {'@id': 'CPS5007'}, {'@id': 'ATS3394'}, {'@id': 'NUR5209'}, {'@id': 'IAR4117'}, {'@id': 'ARC3102'}, {'@id': 'FIT5136'}, {'@id': 'EAE4064'}, {'@id': 'HEC5870'}, {'@id': 'MIC2022'}, {'@id': 'VPR2012'}, {'@id': 'ACX4018'}, {'@id': 'BTH3741'}, {'@id': 'MBS4100'}, {'@id': 'BFX5860'}, {'@id': 'ACS2760'}, {'@id': 'ATS3859'}, {'@id': 'BEX6990'}, {'@id': 'ATS3789'}, {'@id': 'APG5390'}, {'@id': 'ATS3974'}, {'@id': 'IAR2411'}, {'@id': 'ATS2653'}, {'@id': 'MPH5302'}, {'@id': 'APG5075'}, {'@id': 'ATS3225'}, {'@id': 'FIT5011'}, {'@id': 'ATS1125'}, {'@id': 'EAE3591'}, {'@id': 'ATS2189'}, {'@id': 'BEH2141'}, {'@id': 'FOR4003'}, {'@id': 'ATS1905'}, {'@id': 'BTH4280'}, {'@id': 'MKW3610'}, {'@id': 'MPH5203'}, {'@id': 'LAW4184'},

In [14]:
'''define a function to add information of each units in one unit dictionary
    Since the extracted data of each unit is a dict and all the information of units are in a list
    so we combine all the information of one unit and store them into a list by a function
'''


def store_json(part,name):
    if len(unitinform) == len(part):
        for i in range(len(unitinform)):
            unitinform[i][name]=part[i][name]
                        

store_json(title,'title')
store_json(synop,'synopsis')
store_json(prerequisites,'pre_requistics')
store_json(prohibit,'prohibisions')     
store_json(Requirements,'requirements')
store_json(outcomes,'outcomes')
store_json(examiners,'chief_examiners')




#print(unitinform)
print(len(unitinform))

400


In [15]:
##Store data into json form
'''
Since the format of list unitinform is similarly ,so transmit it to json format
'''
unit_dict={}
unit_dict["unit"]=unitinform

final_infor={}
final_infor["units"]=unit_dict

a = open("units_inform.json",'w')
a.write(json.dumps(final_infor, indent=4 ))
a.close()

To XML format

In [16]:
#store data into xml form
'''
According the format of xml write tag before value and add tags and their value into a list
then transmit them into xml
'''

keylist=[]
keylist.append("<units>")
for i in range(len(unitinform)):
    
    for key,value in unitinform[i].items():
        #print(key,value)
        #if key in keys:
        if key=="@id":
            keylist.append('<unit id="' + value +'">')
#print(keylist)

        if key == 'title':
            keylist.append('<title>')
            keylist.append(value)
            keylist.append("</title>")
#print(keylist)

        if key == "synopsis":
            keylist.append("<synopsis>")
            keylist.append(value)
            keylist.append("</synopsis>")
        #pre_requistic

        if key == "pre_requistics":
            if isinstance(value,dict):
                if isinstance(value["pre_requistic"],list):
                    keylist.append("<pre_requistics>")
                    for keys, values in value.items():
                        for j in range(len(values)):
                            keylist.append("<pre_requistic>" + values[j] +"</pre_requistic>")
                    keylist.append("</pre_requistics>")
                if isinstance(value["pre_requistic"],str):
                    keylist.append("<pre_requistics>")
                    keylist.append("<pre_requistic>" + value["pre_requistic"] +"</pre_requistic>")
                    keylist.append("</pre_requistics>")
                
            else:
                keylist.append("<pre_requistics>" + value +"</pre_requistics>")

            

   
         # prohibitions
        if key == "prohibisions":
            if isinstance(value,dict):
                keylist.append("<prohibisions>")
                for keys,values in value.items():
                    for j in range(len(values)):
                        keylist.append("<prohibisions>" + values[j] + "</prohibisions>")
                keylist.append("</prohibisions>")
            else:
                keylist.append("<prohibisions>" + value+ "</prohibisions>")

        #requirement
        if key == "requirements":
            if isinstance(value,dict):
                if isinstance(value["requirement"],list):
                    keylist.append("<requirements>")
                    for keys, values in value.items():
                        for j in range(len(values)):
                            keylist.append("<requirement>" + values[j] +"</requirement>")
                    keylist.append("</requirements>")        
                if isinstance(value["requirement"],str):
                    keylist.append("<requirements>")
                    keylist.append("<requirement>" + value["requirement"] +"</requirement>")
                    keylist.append("</requirements>")
                
            else:
                keylist.append("<requirements>" + value +"</requirements>")

            
        
        
    

        #outcome
        
        if key == "outcomes":
            if isinstance(value,dict):
                if isinstance(value["outcome"],list):
                    keylist.append("<outcomes>")
                    for keys, values in value.items():
                        for j in range(len(values)):
                            keylist.append("<outcome>" + values[j] +"</outcome>")
                    keylist.append("</outcomes>")
                if isinstance(value["outcome"],str):
                    keylist.append("<outcomes>")
                    keylist.append("<outcome>" + value["outcome"] +"</outcome>")
                    keylist.append("</outcomes>")
                
            else:
                keylist.append("<outcomes>" + value +"</outcomes>")



            
        #chief_examiner
        if key == "chief_examiners":
            if isinstance(value,dict):
                if isinstance(value["chief_examiner"],list):
                    keylist.append("<chief_examiners>")
                    for keys, values in value.items():
                        for j in range(len(values)):
                            keylist.append("<chief_examiner>" + values[j] +"</chief_examiner>")
                    keylist.append("</chief_examiners>")
                if isinstance(value["chief_examiner"],str):
                    keylist.append("<chief_examiners>")
                    keylist.append("<chief_examiner>" + value["chief_examiner"] +"</chief_examiner>")
                    keylist.append("</chief_examiners>")
                
            else:
                keylist.append("<chief_examiners>" + value +"</chief_examiners>")


            
    keylist.append("</unit>")
keylist.append("</units>")

print(keylist)

encodeLine = '<?xml version="1.0" encoding="ISO-8859-1"?>\n'
fileObject = open('units_inform.xml', 'w')
fileObject.write(encodeLine)
for unit in keylist:
    fileObject.write(unit)
    fileObject.write('\n')
fileObject.close()

['<units>', '<unit id="ATS1020">', '<title>', 'Leadership for social change 1.', '</title>', '<synopsis>', 'This unit introduces students to interdisciplinary approaches to addressing the key challenges facing the global community. Students are introduced to historical and contemporary concepts, ideas, and frameworks necessary for critically reflecting on the impact of globalisation across cultures. Key to this reflection will be the development of skills in understanding institutions, cultures and people across a range of local, regional, and global contexts. The unit will enable students to identify, define and describe key global challenges and to apply the interdisciplinary knowledge gained in this unit  to critique and construct knowledge in this area.', '</synopsis>', '<pre_requistics>NA</pre_requistics>', '<prohibisions>NA</prohibisions>', '<requirements>', '<requirement>Within semester assessment: 100%</requirement>', '</requirements>', '<outcomes>', '<outcome>Demonstrate an un